# RusTorch CoreML 集成 - Python 绑定

这个笔记本演示了如何通过 Python 绑定使用 RusTorch 的 CoreML 功能。

## 设置和导入

In [ ]:
# 导入 RusTorch Python 绑定
try:
    import rustorch
    print(f"✅ RusTorch 版本: {rustorch.__version__}")
    print(f"📝 描述: {rustorch.__description__}")
    print(f"👥 作者: {rustorch.__author__}")
except ImportError as e:
    print(f"❌ 导入 RusTorch 失败: {e}")
    print("请使用 maturin develop 构建")
    exit(1)

import numpy as np
import platform

print(f"🖥️ 平台: {platform.system()} {platform.release()}")
print(f"🐍 Python 版本: {platform.python_version()}")

## 检查 CoreML 可用性

In [ ]:
# 检查 CoreML 功能
try:
    # 检查 CoreML 是否可用
    coreml_available = rustorch.is_coreml_available()
    print(f"🍎 CoreML 可用: {coreml_available}")
    
    if coreml_available:
        print("🎉 CoreML 可用！")
        
        # 获取设备信息
        device_info = rustorch.get_coreml_device_info()
        print("📱 CoreML 设备信息:")
        print(device_info)
    else:
        print("⚠️ CoreML 不可用")
        if platform.system() != "Darwin":
            print("CoreML 仅在 macOS 上可用")
        else:
            print("CoreML 功能可能未启用")
            
except AttributeError:
    print("❌ 未找到 CoreML 函数")
    print("可能未使用 CoreML 功能构建")
    coreml_available = False
except Exception as e:
    print(f"❌ 检查 CoreML 时出错: {e}")
    coreml_available = False

## CoreML 设备创建和操作

In [ ]:
if coreml_available:
    try:
        # 创建 CoreML 设备
        device = rustorch.CoreMLDevice(device_id=0)
        print(f"🖥️ CoreML 设备已创建: {device}")
        
        # 获取设备信息
        print(f"🆔 设备 ID: {device.device_id()}")
        print(f"✅ 可用: {device.is_available()}")
        print(f"💾 内存限制: {device.memory_limit()} 字节")
        print(f"🧮 计算单元限制: {device.compute_units_limit()}")
        print(f"📚 模型缓存大小: {device.model_cache_size()}")
        
        # 缓存清理
        device.cleanup_cache()
        print("🧹 缓存已清理")
        
    except Exception as e:
        print(f"❌ CoreML 设备操作错误: {e}")
else:
    print("⚠️ 跳过设备操作，因为 CoreML 不可用")

## CoreML 后端配置

In [ ]:
if coreml_available:
    try:
        # 创建 CoreML 后端配置
        config = rustorch.CoreMLBackendConfig(
            enable_caching=True,
            max_cache_size=200,
            enable_profiling=True,
            auto_fallback=True
        )
        print(f"⚙️ 后端配置: {config}")
        
        # 检查和修改配置值
        print(f"📊 启用缓存: {config.enable_caching}")
        print(f"🗂️ 最大缓存大小: {config.max_cache_size}")
        print(f"📈 启用性能分析: {config.enable_profiling}")
        print(f"🔄 自动回退: {config.auto_fallback}")
        
        # 修改配置
        config.enable_profiling = False
        config.max_cache_size = 150
        print(f"\n🔧 更新后的配置: {config}")
        
        # 创建 CoreML 后端
        backend = rustorch.CoreMLBackend(config)
        print(f"🚀 CoreML 后端: {backend}")
        print(f"✅ 后端可用: {backend.is_available()}")
        
        # 获取后端统计信息
        stats = backend.get_stats()
        print(f"📊 后端统计: {stats}")
        print(f"   总操作数: {stats.total_operations}")
        print(f"   缓存命中: {stats.cache_hits}")
        print(f"   缓存未命中: {stats.cache_misses}")
        print(f"   回退操作: {stats.fallback_operations}")
        print(f"   缓存命中率: {stats.cache_hit_rate():.2%}")
        print(f"   回退率: {stats.fallback_rate():.2%}")
        print(f"   平均执行时间: {stats.average_execution_time_ms:.2f}毫秒")
        
        # 缓存清理
        backend.cleanup_cache()
        print("\n🧹 后端缓存已清理")
        
    except Exception as e:
        print(f"❌ CoreML 后端操作错误: {e}")
else:
    print("⚠️ 跳过后端操作，因为 CoreML 不可用")

## 基本张量操作 (CPU)

为了与 CoreML 进行比较，我们首先在 CPU 上执行基本操作。

In [ ]:
try:
    # 基本张量创建和操作
    print("🧮 基本张量操作 (CPU)")
    
    # 从 NumPy 数组创建张量（简化接口）
    data_a = np.random.randn(2, 3).astype(np.float32)
    data_b = np.random.randn(3, 2).astype(np.float32)
    
    print(f"📐 矩阵 A 形状: {data_a.shape}")
    print(f"📐 矩阵 B 形状: {data_b.shape}")
    
    # 使用 NumPy 进行矩阵乘法（用于比较）
    numpy_result = np.matmul(data_a, data_b)
    print(f"✅ NumPy matmul 结果形状: {numpy_result.shape}")
    print(f"📊 结果（前几个元素）: {numpy_result.flatten()[:4]}")
    
    print("\n🚀 CPU 操作完成")
    
except Exception as e:
    print(f"❌ 张量操作错误: {e}")

## 性能比较模拟

In [ ]:
import time

def benchmark_matrix_operations():
    """比较不同矩阵大小的性能"""
    
    sizes = [(64, 64), (128, 128), (256, 256), (512, 512)]
    
    print("🏁 性能比较:")
    print("大小\t\tCPU 时间 (毫秒)\t预期 CoreML (毫秒)")
    print("-" * 50)
    
    for size in sizes:
        # 测量 CPU 执行时间
        a = np.random.randn(*size).astype(np.float32)
        b = np.random.randn(size[1], size[0]).astype(np.float32)
        
        start_time = time.time()
        result = np.matmul(a, b)
        cpu_time = (time.time() - start_time) * 1000
        
        # 预期 CoreML 时间（假设）
        # 在实际实现中，使用 CoreML 后端的实际测量值
        expected_coreml_time = cpu_time * 0.6  # 假设：CoreML 快 40%
        
        print(f"{size[0]}x{size[1]}\t\t{cpu_time:.2f}\t\t{expected_coreml_time:.2f}")

benchmark_matrix_operations()

print("\n📝 注意：CoreML 时间是假设的。实际值取决于具体实现。")

## 设备选择模拟

In [ ]:
def simulate_device_selection():
    """模拟智能设备选择"""
    
    operations = [
        ("小矩阵乘法", (16, 16), "CPU"),
        ("中等矩阵乘法", (128, 128), "Metal GPU"),
        ("大矩阵乘法", (512, 512), "CoreML" if coreml_available else "Metal GPU"),
        ("激活函数", (32, 64, 128, 128), "Metal GPU"),
        ("小卷积", (1, 3, 32, 32), "CPU"),
        ("大卷积", (16, 64, 224, 224), "CoreML" if coreml_available else "Metal GPU"),
        ("复数运算", (128, 128), "Metal GPU"),  # CoreML 不支持
        ("统计分布", (1000,), "CPU"),  # CoreML 不支持
    ]
    
    print("🎯 智能设备选择模拟:")
    print("操作\t\t\t张量形状\t\t选择的设备")
    print("-" * 60)
    
    for name, shape, device in operations:
        shape_str = "x".join(map(str, shape))
        print(f"{name:<23}\t{shape_str:<15}\t{device}")
    
    print("\n📝 选择逻辑:")
    print("  • 小操作: CPU（避免开销）")
    print("  • 中等操作: Metal GPU（平衡）")
    print("  • 大操作: CoreML（优化）")
    print("  • 不支持的操作: GPU/CPU 回退")

simulate_device_selection()

## 实用示例：简单神经网络层

In [ ]:
def simulate_neural_network_layer():
    """模拟神经网络层"""
    
    print("🧠 神经网络层模拟:")
    
    # 批次大小和层配置
    batch_size = 32
    input_features = 784  # 28x28 MNIST
    hidden_features = 256
    output_features = 10  # 10个类别
    
    print(f"📊 批次大小: {batch_size}")
    print(f"🔢 输入特征: {input_features}")
    print(f"🧮 隐藏特征: {hidden_features}")
    print(f"🎯 输出特征: {output_features}")
    
    # 前向传播模拟
    steps = [
        ("输入 → 隐藏", f"({batch_size}, {input_features}) @ ({input_features}, {hidden_features})", "CoreML" if coreml_available else "Metal"),
        ("ReLU 激活", f"({batch_size}, {hidden_features})", "Metal"),
        ("隐藏 → 输出", f"({batch_size}, {hidden_features}) @ ({hidden_features}, {output_features})", "CoreML" if coreml_available else "Metal"),
        ("Softmax", f"({batch_size}, {output_features})", "CPU"),
    ]
    
    print("\n🔄 前向传播模拟:")
    total_time = 0
    
    for step, shape, device in steps:
        # 虚拟执行时间（毫秒）
        if device == "CoreML":
            time_ms = np.random.uniform(0.5, 2.0)
        elif device == "Metal":
            time_ms = np.random.uniform(1.0, 3.0)
        else:  # CPU
            time_ms = np.random.uniform(0.2, 1.0)
        
        total_time += time_ms
        print(f"  {step:<15} {shape:<30} {device:<8} {time_ms:.2f}毫秒")
    
    print(f"\n⏱️ 总前向传播时间: {total_time:.2f}毫秒")
    print(f"🚀 估计吞吐量: {1000/total_time:.0f} 批次/秒")

simulate_neural_network_layer()

## 总结和后续步骤

In [ ]:
print("📋 RusTorch CoreML 集成总结:")
print()
print("✅ 已完成项目:")
print("  • Jupyter 环境设置")
print("  • Rust 内核和 Python 绑定创建")
print("  • CoreML 可用性检查")
print("  • 设备管理和配置")
print("  • 后端统计和性能分析")
print("  • 智能设备选择")
print()
print("🚧 未来开发:")
print("  • 实际 CoreML 操作实现")
print("  • 性能基准测试")
print("  • 更多激活函数和层类型")
print("  • 错误处理改进")
print("  • 内存优化")
print()
print("🎯 推荐的后续步骤:")
print("  1. 加载和测试真实的 CoreML 模型")
print("  2. 比较 Metal 和 CoreML 性能")
print("  3. 使用真实深度学习工作流测试")
print("  4. 在生产环境中评估")

if coreml_available:
    print("\n🎉 恭喜！CoreML 可用，所有功能都可以测试。")
else:
    print("\n⚠️ CoreML 不可用，但基本功能正常工作。")
    print("   我们建议在 macOS 上启用 CoreML 功能进行构建。")